In [1]:
from itertools import chain
from math import floor
import sys

from scipy import integrate
sys.path.append("/home/m/murray/dtolgay/scratch")
from tools import constants

import numpy as np 
import pandas as pd 
import os

from scipy.spatial import KDTree
from scipy.interpolate import LinearNDInterpolator, NearestNDInterpolator

from time import time 
from concurrent.futures import ProcessPoolExecutor

from lichenVolumeIntegral_smoothingLength import read_cloudy_gas_particles, read_training_data, split_dataframe
from lichenVolumeIntegral_smoothingLength import find_converged_run

In [2]:

# Global variables
epsilon = 1e-30
# mu = 1.2
mu = 1.38 # Krumholz and Gnedin 


In [3]:
galaxy_name = "gal4"
galaxy_type = "firebox"
redshift = "0.0"
max_workers = 1
write_interpolator_info=False

directory_name = "voronoi_1e6"
# directory_name = "voronoi_1e5"

print(
    f"------------------------------------------ {galaxy_name} ------------------------------------------"
)

## Check if file exits. If it exists do not continue running the code, if not run the code.
cloudy_gas_particles_file_directory = f"/home/m/murray/dtolgay/scratch/post_processing_fire_outputs/skirt/runs_hden_radius/{galaxy_type}/z{redshift}/{galaxy_name}/{directory_name}"
# cloudy_gas_particles_file_directory = f"/home/m/murray/dtolgay/scratch/cloudy_runs/z_3/m12f_res7100_md_test"

write_file_path = f"{cloudy_gas_particles_file_directory}/L_line_smoothingLength_lichenTypeCalculation.txt"

print("\n")
if os.path.isfile(write_file_path):
    print(f"{write_file_path} exits, the code is stopped.")
else: 
    print(f"{write_file_path} doen't exist. Continuing....")


# Read gas particles 
gas_particles_df, gas_column_names = read_cloudy_gas_particles(cloudy_gas_particles_file_directory)

gas_particles_df = gas_particles_df.iloc[0:10].copy() #TODO: Delete 

base_line_names = [
    "ly_alpha",
    "h_alpha",
    "h_beta",
    "co_10",
    "co_21",
    "co_32",
    "co_43",
    "co_54",
    "co_65",
    "co_76",
    "co_87",
    "13co",
    "c2",
    "o3_88",
    "o3_5006",
    "o3_4958",        
]

################ Read training data particles 

# 1st set of run
train_data_base_file_dir_1 = "/scratch/m/murray/dtolgay/cloudy_runs/z_0"
train_data_main_directory_1 = "cr_1_CO87_CII_H_O3/cr_1_CO87_CII_H_O3_metallicity_above_minus_2" 

train_data_df_1, line_names_with_log = read_training_data(
    base_file_dir = train_data_base_file_dir_1, 
    main_directory = train_data_main_directory_1, 
    file_name = "I_line_values_without_reversing.txt", 
    base_line_names = base_line_names
)    

# 2nd set of run
train_data_base_file_dir_2 = "/scratch/m/murray/dtolgay/cloudy_runs/z_0"
train_data_main_directory_2 = "cr_1_CO87_CII_H_O3/cr_1_CO87_CII_H_O3_metallicity_minus2_minus3point5" 

train_data_df_2, line_names_with_log = read_training_data(
    base_file_dir = train_data_base_file_dir_2, 
    main_directory = train_data_main_directory_2, 
    file_name = "I_line_values_without_reversing.txt", 
    base_line_names = base_line_names
)    

train_data_file_paths = [f"{train_data_base_file_dir_1}/{train_data_main_directory_1}", f"{train_data_base_file_dir_2}/{train_data_main_directory_2}"]

# Concattanete two dataframes 
train_data_df = pd.concat([train_data_df_2, train_data_df_1])

########

# train_data_base_file_dir_test = "/home/m/murray/dtolgay/scratch/cloudy_runs/z_3/"
# train_data_main_director_test = "m12f_res7100_md_test"
# train_data_df, line_names_with_log = read_training_data(
#     base_file_dir = train_data_base_file_dir_test, 
#     main_directory = train_data_main_director_test, 
#     file_name = "I_line_values_without_reversing.txt", 
#     base_line_names = base_line_names
# )   

# train_data_file_paths = [f"{train_data_base_file_dir_test}/{train_data_main_director_test}"]

# Split dataframe into several dataframes to run the parallely. 
gas_particles_df_chunks = split_dataframe(
        df=gas_particles_df,
        max_workers=max_workers, 
    )


------------------------------------------ gal4 ------------------------------------------


/home/m/murray/dtolgay/scratch/post_processing_fire_outputs/skirt/runs_hden_radius/firebox/z0.0/gal4/voronoi_1e6/L_line_smoothingLength_lichenTypeCalculation.txt doen't exist. Continuing....
/home/m/murray/dtolgay/scratch/post_processing_fire_outputs/skirt/runs_hden_radius/firebox/z0.0/gal4/voronoi_1e6/cloudy_gas_particles.txt read and dataframe is created!
Training data is started to be read.
Dropping NaN containing lines
Check if all intensities are positive. Then set 0 values to 1e-30
Not all intensities are are non-negative. Setting them to epsilon
/scratch/m/murray/dtolgay/cloudy_runs/z_0/cr_1_CO87_CII_H_O3/cr_1_CO87_CII_H_O3_metallicity_above_minus_2/I_line_values_without_reversing.txt is read.
Training data is started to be read.


/gpfs/fs0/scratch/m/murray/dtolgay/post_processing_fire_outputs/skirt/python_files/analyze_hden_metallicity_turbulence_isrf_radius/lichen_type_calculation/lichenVolumeIntegral_smoothingLength.py:355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_df['log_column_density'] = np.log10(


Dropping NaN containing lines
Check if all intensities are positive. Then set 0 values to 1e-30
Not all intensities are are non-negative. Setting them to epsilon
/scratch/m/murray/dtolgay/cloudy_runs/z_0/cr_1_CO87_CII_H_O3/cr_1_CO87_CII_H_O3_metallicity_minus2_minus3point5/I_line_values_without_reversing.txt is read.


/gpfs/fs0/scratch/m/murray/dtolgay/post_processing_fire_outputs/skirt/python_files/analyze_hden_metallicity_turbulence_isrf_radius/lichen_type_calculation/lichenVolumeIntegral_smoothingLength.py:355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_df['log_column_density'] = np.log10(


In [35]:
def calculate_L_line_with_volume_integral_per_gas_particle(train_data_file_paths, fdir, log_cloudy_metallicity, log_gas_radius_in_pc):
    
    COLUMNS_EMISSIVITY = [
        "radius",
        "ly_alpha",
        "h_alpha",
        "h_beta",
        "CO10",
        "CO21",
        "CO32",
        "CO43",
        "CO54",
        "CO65",
        "CO76",
        "CO87",
        "13CO",
        "C2",
        "O3_88um",
        "O3_5006um",
        "O3_4958um",
    ]

    if log_cloudy_metallicity > -1.9:
        TRAIN_DATA_FILE_PATH = train_data_file_paths[0]
    else:
        TRAIN_DATA_FILE_PATH = train_data_file_paths[1]
        
    with open(f"{TRAIN_DATA_FILE_PATH}/{fdir}/{fdir}.out", "r") as file:
        lines = file.readlines()
        last_line = lines[-1]    

        if "OK" == last_line[len(last_line) - 4 : len(last_line) - 2]:
            cloudy_em_str = np.loadtxt(
                fname=f"{TRAIN_DATA_FILE_PATH}/{fdir}/{fdir}_em.str"
            )
            # Only use the coverged run
            cloudy_em_str = find_converged_run(cloudy_em_str=cloudy_em_str)
            
            # Create a dictionary to store separate arrays for each column
            emissivity_arrays = pd.DataFrame(cloudy_em_str, columns=COLUMNS_EMISSIVITY)
            
            ### use only the radius values smaller than the radius of gas particle
            # Convert gas length to cm 
            gas_shielding_length_in_cm = 10**log_gas_radius_in_pc * constants.pc2cm
            condition = emissivity_arrays['radius'] < gas_shielding_length_in_cm
            emissivity_arrays = emissivity_arrays.loc[condition].copy()            
            
            ### Get the volume integrals 
            volume_integrals = {}
            for key in emissivity_arrays:
                if key != "radius":
                    volume_integrals[key] = 4 * np.pi * (
                        integrate.trapz(y=emissivity_arrays[key]*emissivity_arrays['radius']**2, x=emissivity_arrays['radius'])
                    ) # erg s^-1      
                
    print("Finished")
    return volume_integrals 

def calculate_Lline(gas_particles_df, train_data_df):
    
    # Initiate array to store the index of gas and luminosities 
    gas_indices_luminosities = []
    
    #### Tree creation
    ##  Define centers
    train_data_column_names = ["log_metallicity", "log_hden", "log_turbulence", "log_isrf"]
    ## Only use the columns with these column names 
    train_data_df = train_data_df[train_data_column_names].drop_duplicates()
    ## Create the tree
    tree = KDTree(
        train_data_df,
    ) # Create a tree for the training data


    gas_data_tree_query_columns = train_data_column_names # This can change in the future
    shilding_length_name = "log_smoothing_length"    
    
    for indices, gas in gas_particles_df.iloc[0:1].iterrows():
        # Find the closest cloudy runs 
        distances, cloudy_index = tree.query(gas[gas_data_tree_query_columns], k=1) # Return 1 but mostly everything should be done in the first iteration

        # Get the cloudy center 
        cloudy_center = train_data_df.iloc[cloudy_index].copy()

        # Loop through the used radius values 
        log_radius_used_for_cloudy_run_array = [5, 4.5, 4, 3.5, 3, 2.5, 2, 1.5, 1]
        for log_radius_cloudy_run in log_radius_used_for_cloudy_run_array:
            fdir = f"hden{cloudy_center['log_hden']:.5f}_metallicity{cloudy_center['log_metallicity']:.5f}_turbulence{cloudy_center['log_turbulence']:.5f}_isrf{cloudy_center['log_isrf']:.5f}_radius{log_radius_cloudy_run:.5f}"
            try:
                volume_integrals = calculate_L_line_with_volume_integral_per_gas_particle(
                    train_data_file_paths = train_data_file_paths, 
                    fdir = fdir, 
                    log_cloudy_metallicity = cloudy_center['log_metallicity'], 
                    log_gas_radius_in_pc = gas[shilding_length_name]
                )
                
                luminosities = volume_integrals.values() # erg/s 

                break 
            except Exception as e:
                print(f"Exception occured. log_shielding_length = {log_radius_cloudy_run} didn't work. Trying other radius values. Error is: \n {e}")
                luminosities = np.array(len(line_names_with_log)) * np.nan # Create nan array


                
        # Ensure gas['index'] is a 1-dimensional array
        gas_index = np.array([gas['index']])  # Wrap scalar in a list to make it 1D

        # Ensure luminosities is a 1-dimensional array
        luminosities = np.atleast_1d(luminosities)  # Convert to 1D if it's not

        # Concatenate the arrays
        gas_indices_luminosities.append(
            np.concatenate((gas_index, luminosities))
        )    
    
    return gas_indices_luminosities 

calculate_Lline(
    gas_particles_df = gas_particles_df, 
    train_data_df = train_data_df
)


Finished


[array([0.0,
        dict_values([2.2792319325723668e+35, 1.0185917501652731e+36, 3.1603137313801025e+35, 8.280209618648293e+28, 9.580488286214246e+29, 1.5875389333316487e+30, 8.799707677525588e+29, 2.129605543819007e+29, 2.5522341088921896e+28, 1.8582257546918336e+27, 2.7909176092453716e+26, 2.2623466919017751e+27, 5.422234277293409e+37, 1.7495932623742852e+31, 2.5411893457829148e+29, 8.517221539733531e+28])],
       dtype=object)]

{'ly_alpha': 2.2792319325723668e+35, 'h_alpha': 1.0185917501652731e+36, 'h_beta': 3.1603137313801025e+35, 'CO10': 8.280209618648293e+28, 'CO21': 9.580488286214246e+29, 'CO32': 1.5875389333316487e+30, 'CO43': 8.799707677525588e+29, 'CO54': 2.129605543819007e+29, 'CO65': 2.5522341088921896e+28, 'CO76': 1.8582257546918336e+27, 'CO87': 2.7909176092453716e+26, '13CO': 2.2623466919017751e+27, 'C2': 5.422234277293409e+37, 'O3_88um': 1.7495932623742852e+31, 'O3_5006um': 2.5411893457829148e+29, 'O3_4958um': 8.517221539733531e+28}
Exception occured. log_shielding_length = 5 didn't work. Trying other radius values. Error is: 
 'NoneType' object has no attribute 'values'
{'ly_alpha': 2.2655029693382738e+35, 'h_alpha': 9.862028126175826e+35, 'h_beta': 3.0594603561408283e+35, 'CO10': 7.885464407215195e+28, 'CO21': 9.123782467653008e+29, 'CO32': 1.5118570733807148e+30, 'CO43': 8.38020136145811e+29, 'CO54': 2.02809558198812e+29, 'CO65': 2.430737847588642e+28, 'CO76': 1.771171423555921e+27, 'CO87': 2.6

NameError: name 'gas_indices_luminosities' is not defined

In [9]:
import scipy
print(scipy.__version__)


1.5.2


In [8]:
pip install --upgrade scipy --user


  Using cached scipy-1.10.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)
  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 2.2.0 requires scipy==1.4.1; python_version >= "3", but you'll have scipy 1.10.1 which is incompatible.
tensorflow 2.2.0 requires tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.4.0 which is incompatible.
tensorflow 2.2.0 requires tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 2.5.0 which is incompatible.
Note: you may need to restart the kernel to us

In [ ]:
python --version